In [2]:
import pandas as pd
import re
import math
import requests
import json

In [3]:
df = pd.read_csv('test_urls.csv')
df.head()

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,avalanche.nhl.com/club/player.htm?id=8456933,good
3,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
4,mail.printakid.com/www.online.americanexpress....,bad


In [3]:
# https://www.whoisxmlapi.com/whoisserver/WhoisService?apiKey=at_BGLU4RpM0fcKHpqsqHAYRWFl4bIt2&domainName=facebook.com

# API Key: at_BGLU4RpM0fcKHpqsqHAYRWFl4bIt2
# Rapid API Key: d2592fe48dmsh190428470dadf9cp1bf762jsnf40816e6115e (Shauryansingh23102001@gmail.com)

def whois_api_request(domain_url):
    url = "https://whoisapi-whois-v2-v1.p.rapidapi.com/whoisserver/WhoisService"

    headers = {
	    "X-RapidAPI-Key": "d2592fe48dmsh190428470dadf9cp1bf762jsnf40816e6115e",
	    "X-RapidAPI-Host": "whoisapi-whois-v2-v1.p.rapidapi.com"
    }

    querystring = {
        "domainName":domain_url,
        "apiKey":"at_BGLU4RpM0fcKHpqsqHAYRWFl4bIt2",
        "outputFormat":"JSON",
        "da":"0",
        "ipwhois":"1",
        "thinWhois":"0",
        "_parse":"0",
        "preferfresh":"1",
        "checkproxydata":"0",
        "ip":"1"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    # response_dict = {"JSON_OBJ": response.json(), "STRING_OBJ": response_text}
    return response

In [4]:
def count_digits(word):
  return len(re.findall('[0-9]', word))

def count_alphabets(word):
  return len(re.findall('[A-z]', word))

def remove_special_characters(input_string):
  # Pattern to match any non-alphanumeric and non-space character
  pattern = r'[^a-zA-Z0-9\s]'  
  return re.sub(pattern, '', input_string)

def calculate_entropy(word):
  if len(word) == 0:
    return 0

  entropy = 0
  word_length = len(word)
  char_count = {}

  for char in word:
    if char in char_count:
      char_count[char] += 1
    else:
      char_count[char] = 1

  for char in char_count:
    probability = char_count[char] / word_length
    entropy += -probability * math.log2(probability)

  return round(entropy, 2)

In [5]:
def count_special_characters(input_string):
  special_characters = "!@#$%^&*()-_=+[]{}|;:',.<>/?"
  count = 0

  for char in input_string:
    if char not in special_characters and not char.isalnum():
      count += 1

  return count


In [6]:
DATA = []

In [7]:
# Remember you have to write logic to check if the data returned by who is api has Data error or missing fields
for index, row in df.iterrows():

  if(index == 5):
    break

  URL = row["URL"]
  label = row["Label"]

  domain = URL.split("/")[0]
  url_length = len(URL)
  domain_entropy = calculate_entropy(domain)

  temp = URL.split("/")
  subdomain = temp[0].split(".")[-1]
  path_rest = "/".join(temp[1:])
  path_rest_length = len(path_rest)

  words_list = re.split("[/,.,?,-,+,=,&,~,!,@,#,$,%,_,;]", path_rest)

  tld = domain.split(".")[-1]
  tld_length = len(tld)
  tld = tld.split(":")[0]
  tld = tld.split("-")[0]

  num_spcs_chars = count_special_characters(path_rest)

  url = "https://whoisapi-whois-v2-v1.p.rapidapi.com/whoisserver/WhoisService"

  # domain_url_name = "youtube.com"

  querystring = {
    "domainName":domain,
    "apiKey":"at_BGLU4RpM0fcKHpqsqHAYRWFl4bIt2",
    "outputFormat":"JSON",
    "da":"0",
    "ipwhois":"1",
    "thinWhois":"0",
    "_parse":"0",
    "preferfresh":"1",
    "checkproxydata":"0",
    "ip":"1"
    }

  headers = {
    "X-RapidAPI-Key": "d2592fe48dmsh190428470dadf9cp1bf762jsnf40816e6115e",
    "X-RapidAPI-Host": "whoisapi-whois-v2-v1.p.rapidapi.com"
  }

  response = requests.get(url, headers=headers, params=querystring)

  status_code = response.status_code
  response_data = response.json()
  
  if "WhoisRecord" in response_data:
    if "createdDate" in response_data["WhoisRecord"]:
      created_year = int(response_data["WhoisRecord"]["createdDate"][0:4])
    elif "registryData" in response_data["WhoisRecord"] and "createdDate" in response_data["WhoisRecord"]["registryData"]:
      created_year = int(response_data["WhoisRecord"]["registryData"]["createdDate"][0:4])
    else:
      created_year = None

  if "WhoisRecord" in response_data:
    if "updatedDate" in response_data["WhoisRecord"]:
      updated_year = int(response_data["WhoisRecord"]["updatedDate"][0:4])
    elif "registryData" in response_data["WhoisRecord"] and "updatedDate" in response_data["WhoisRecord"]["registryData"]:
      updated_year = int(response_data["WhoisRecord"]["registryData"]["updatedDate"][0:4])
    else:
      updated_year = None

  if "WhoisRecord" in response_data:
    if "expiresDate" in response_data["WhoisRecord"]:
      expires_year = int(response_data["WhoisRecord"]["expiresDate"][0:4])
    elif "registryData" in response_data["WhoisRecord"] and "expiresDate" in response_data["WhoisRecord"]["registryData"]:
      expires_year = int(response_data["WhoisRecord"]["registryData"]["expiresDate"][0:4])
    else:
      expires_year = None

  if "WhoisRecord" in response_data:
    if ("registrant" in response_data["WhoisRecord"] and 
      "countryCode" in response_data["WhoisRecord"]["registrant"]):
      country_code = response_data["WhoisRecord"]["registrant"]["countryCode"]
    elif (
      "registryData" in response_data["WhoisRecord"] and
      "registrant" in response_data["WhoisRecord"]["registryData"] and
      "countryCode" in response_data["WhoisRecord"]["registryData"]["registrant"]):
      country_code = response_data["WhoisRecord"]["registryData"]["registrant"]["countryCode"]
    else:
      country_code = None
      
  if "WhoisRecord" in response_data and "estimatedDomainAge" in response_data["WhoisRecord"]:
    domain_age = int(response_data["WhoisRecord"]["estimatedDomainAge"])
  else:
    domain_age = None

  df2_data = [
    subdomain,
    domain,
    tld,
    path_rest,
    url_length,
    path_rest_length,
    num_spcs_chars,
    domain_entropy,
    domain_age,
    created_year,
    updated_year,
    expires_year,
    country_code,
  ]

  DATA.append(df2_data)
df2 = pd.DataFrame(DATA, columns=["subdomain", "domain", "tld", "path_rest", "url_length", "path_rest_length", "num_spcs_chars", "domain_entropy", "domain_age", "created_year", "updated_year", "expires_year", "country_code"])


In [ ]:
# Assuming you have a file named 'data.json' containing the JSON data
with open('output.json') as file:
  parsed_data = json.load(file)

print(parsed_data["WhoisRecord"]["createdDate"][0:4])

created_year = int(parsed_data["WhoisRecord"]["createdDate"][0:4])
updated_year = int(parsed_data["WhoisRecord"]["updatedDate"][0:4])
expires_year = int(parsed_data["WhoisRecord"]["expiresDate"][0:4])
country_code = parsed_data["WhoisRecord"]["registrant"]["countryCode"]
domain_age = int(parsed_data["WhoisRecord"]["estimatedDomainAge"])

In [ ]:
# INSERT COLUMNS IN BETWEEN THE DATAFRAME
insert_position = df.columns.get_loc('URL') + 1

for col_name in df2.columns[::-1]:
  df.insert(insert_position, col_name, df2[col_name])

In [ ]:
result_horizontal = pd.concat([df1, df2], axis=1)
print(result_horizontal)

In [11]:
df.to_csv('test_csv.csv', index=False)